In [ ]:
!pip install fastapi uvicorn nest-asyncio python-multipart timm opencv-python pyngrok

!git clone https://github.com/isl-org/MiDaS.git

!wget https://huggingface.co/Intel/dpt-large/resolve/main/dpt_large-midas-2f21e586.pt -O /content/dpt_large-midas.pt


In [ ]:
import sys
sys.path.append('/content/MiDaS')


In [ ]:
import torch
import cv2
import numpy as np
from torchvision.transforms import Compose
import nest_asyncio
import uvicorn
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import Response

# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load MiDaS model
model_type = "MiDaS_small"  # options: "DPT_Large", "DPT_Hybrid", "MiDaS_small"
midas = torch.hub.load("intel-isl/MiDaS", model_type)
midas.to(device)
midas.eval()

# Load transforms
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
transform = midas_transforms.dpt_transform

In [ ]:
app = FastAPI()

@app.post("/predict")
async def predict_depth(image: UploadFile = File(...)):
    contents = await image.read()
    img_np = np.frombuffer(contents, np.uint8)
    img = cv2.imdecode(img_np, cv2.IMREAD_COLOR)

    # Preprocess
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    input_tensor = transform(img_rgb).to(device)

    # Predict
    with torch.no_grad():
        prediction = midas(input_tensor)
        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode="bicubic",
            align_corners=False
        ).squeeze()

    # Postprocess
    depth_map = prediction.cpu().numpy()
    depth_map = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX)
    depth_map = depth_map.astype(np.uint8)
    depth_color = cv2.applyColorMap(depth_map, cv2.COLORMAP_MAGMA)

    _, encoded_img = cv2.imencode('.jpg', depth_color)
    return Response(content=encoded_img.tobytes(), media_type="image/jpeg")

# Allow FastAPI to run in Colab
nest_asyncio.apply()


In [ ]:
!ngrok config add-authtoken <<paste your ngrok token here>>


In [ ]:

from pyngrok import ngrok

public_url = ngrok.connect(7860)
print("🔗 Public URL:", public_url)


In [ ]:
uvicorn.run(app, host="0.0.0.0", port=7860)


In [ ]:
!pkill -f ngrok

